In [1]:
import folium

import ee   # Google EarthEngine API
ee.Initialize()

In [ ]:
# ndvi = ee.Image(image).normalizedDifference(['SREFL_CH2','SREFL_CH1']).rename('NDVI')

# Single Image, NDVI

In [2]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
    ).add_to(self)

In [9]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
    'min': -1,
    'max': 1,
    }
#'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
avhrr_col = ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5')
avhrr_1 = avhrr_col.filter(ee.Filter.date('1985-07-01', '1985-07-05')).first()  # do not select bands

qa = avhrr_1.select('QA')
cldmsk = 1 << 1   # cloud
cldsmsk = 1 << 2  # cloud shadow
mask = (qa.bitwiseAnd(cldmsk).eq(0)).And(qa.bitwiseAnd(cldsmsk).eq(0))  
mask1 = qa.bitwiseAnd(cldmsk).eq(0)
ndvi = avhrr_1.normalizedDifference(['SREFL_CH2','SREFL_CH1']).rename('NDVI')
ndvi_m = ndvi.updateMask(mask)
ndvi_cldmsk = ndvi.updateMask(mask1)

my_map.add_ee_layer(ndvi_m, vis_params, 'AVHRR_NDVI')
my_map.add_ee_layer(ndvi_cldmsk, vis_params, 'AVHRR_NDVI_only_cloudmask')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Use .map on an image collection

In [5]:
def cldmsk(img):
    qa = img.select('QA')
    cldmsk = 1 << 1   # cloud
    cldsmsk = 1 << 2  # cloud shadow
    mask = (qa.bitwiseAnd(cldmsk).eq(0)).And(qa.bitwiseAnd(cldsmsk).eq(0))  
    mask1 = qa.bitwiseAnd(cldmsk).eq(0)
    img = img.updateMask(mask)
    img = img.updateMask(mask1)
    return img

In [8]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
    'min': -1,
    'max': 1,
    }
#'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
avhrr_col = ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5')
avhrr_1 = avhrr_col.filter(ee.Filter.date('1985-07-01', '1985-08-05')).map(cldmsk)  # do not select bands

avhrr_2 = avhrr_1.mean()
ndvi = avhrr_2.normalizedDifference(['SREFL_CH2','SREFL_CH1']).rename('NDVI')

my_map.add_ee_layer(ndvi, vis_params, 'AVHRR_NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

# Manual bandmath

In [12]:
def cldmsk(img):
    qa = img.select('QA')
    cldmsk = 1 << 1   # cloud
    cldsmsk = 1 << 2  # cloud shadow
    mask = (qa.bitwiseAnd(cldmsk).eq(0)).And(qa.bitwiseAnd(cldsmsk).eq(0))  
    mask1 = qa.bitwiseAnd(cldmsk).eq(0)
    img = img.updateMask(mask)
    img = img.updateMask(mask1)
    return img

In [14]:
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
    'min': -1,
    'max': 1,
    }
#'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
avhrr_col = ee.ImageCollection('NOAA/CDR/AVHRR/SR/V5')
avhrr_1 = avhrr_col.filter(ee.Filter.date('1985-07-01', '1985-08-05')).map(cldmsk)  # do not select bands

avhrr_2 = avhrr_1.mean()
bm1 = avhrr_2.expression(
          'float((B2-B3A)/(B2+B3A))',
          {
              'B2':avhrr_2.select(['SREFL_CH2']),
              'B3A': avhrr_2.select(['SREFL_CH3'])
          }).rename('EVI')

my_map.add_ee_layer(bm1, vis_params, 'AVHRR_Bandmath1')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)